<a href="https://colab.research.google.com/github/Cinq-hr/Portfolio-Optimization/blob/main/Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN','BRK-B','TSLA','NVDA','KMX','F']  #

data = yf.download(tickers, start="2020-01-01", end="2023-01-01")['Close']

data.head()

In [ ]:
returns = data.pct_change().dropna()
returns.head()

In [ ]:

mean_returns = returns.mean()

cov_matrix = returns.cov()

In [ ]:

risk_free_rate = 0.02

sharpe_ratio_threshold = 0.7


num_portfolios = 10000
results = np.zeros((3, num_portfolios))
weights_record = []

underperforming_count = 0
top_performing_portfolios = []

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    weights_record.append(weights)


    portfolio_return = np.dot(weights, mean_returns) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 252, weights)))


    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility


    results[0, i] = portfolio_return
    results[1, i] = portfolio_volatility
    results[2, i] = sharpe_ratio


    if sharpe_ratio < sharpe_ratio_threshold:
        underperforming_count += 1


    top_performing_portfolios.append((i, portfolio_return, portfolio_volatility, sharpe_ratio, weights))


top_performing_portfolios = sorted(top_performing_portfolios, key=lambda x: x[3], reverse=True)


top_5_portfolios = top_performing_portfolios[:5]


print(f"Number of underperforming portfolios: {underperforming_count}")

print("\nTop 5 best-performing portfolios:")
for idx, (portfolio_index, portfolio_return, portfolio_volatility, sharpe_ratio, weights) in enumerate(top_5_portfolios):
    print(f"\nPortfolio {portfolio_index + 1}:")
    print(f"  Return: {portfolio_return:.2f}%")
    print(f"  Volatility: {portfolio_volatility:.2f}%")
    print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")


    print("  Stocks and their weights (%):")
    for i, weight in enumerate(weights):
        print(f"    {tickers[i]}: {weight * 100:.2f}%")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(results[1], results[0], c=results[2], cmap='viridis', alpha=0.5)
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(results[1], results[0], c=results[2], cmap='viridis', alpha=0.5)


best_sharpe_idx = np.argmax(results[2])


plt.scatter(results[1][best_sharpe_idx], results[0][best_sharpe_idx], color='red', marker='*', s=200, label='Best Portfolio')


best_sharpe_ratio = results[2][best_sharpe_idx]
print(f"The Sharpe Ratio of the Best Portfolio is: {best_sharpe_ratio:.4f}")

plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.grid(True)
plt.legend(loc='upper left')
plt.show()

In [ ]:
max_sharpe_idx = np.argmax(results[2])


optimal_weights = weights_record[max_sharpe_idx]


print("Best Portfolio Allocation (Max Sharpe Ratio):")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {optimal_weights[i]*100:.2f}%")